Camada Silver – Limpeza e Transformação

##  Documentação

**Objetivo:**  
Limpar, padronizar e enriquecer os dados brutos da camada Bronze, removendo registros inválidos e preparando os dados para análise.

**Entradas:**  
- Tabela Delta: `lakehouse.bronze.yellow_trip`

**Saídas:**  
- Tabela Delta: `lakehouse.silver.yellow_trip` (tratada e enriquecida)

**Transformações aplicadas:**
- Renomeio de colunas para `snake_case`
- Filtros:
  - `passenger_count > 0`
  - `trip_distance > 0`
  - `fare_amount > 0`
- Remoção de duplicatas
- Enriquecimento:
  - `vendor_id` → nome da empresa
  - `payment_type` → descrição da forma de pagamento

## Decisões Tomadas

- Utilizado PySpark para processamento distribuído eficiente.
- Adicionadas colunas descritivas para facilitar análise de negócio.
- Evitou-se qualquer transformação que mudasse os valores originais relevantes.


In [0]:
import pyspark.sql.functions as F

In [0]:
# Reads the bronze layer table
table_bronze = spark.read.table("lakehouse.bronze.yellow_trip")

# Selects and renames columns with snake_case
df_silver = table_bronze.select(
    F.col("VendorID").alias("vendor_id"),
    F.col("tpep_pickup_datetime").alias("pickup_datetime"),
    F.col("tpep_dropoff_datetime").alias("dropoff_datetime"),
    F.col("passenger_count"),
    F.col("trip_distance"),
    F.col("RatecodeID").alias("ratecode_id"),
    F.col("store_and_fwd_flag"),
    F.col("PULocationID").alias("pu_location_id"),
    F.col("DOLocationID").alias("do_location_id"),
    F.col("payment_type"),
    F.col("fare_amount"),
    F.col("total_amount")
)

# Filters invalid records
df_silver = df_silver.filter(
    (F.col("passenger_count") > 0) &
    (F.col("trip_distance") > 0) &
    (F.col("fare_amount") > 0)
)

# Removes duplicates
df_silver_clean = df_silver.dropDuplicates()

In [0]:

df_silver_clean.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouse.silver.yellow_trip")
